In [ ]:
# this notebook is supposed to be run on colab

# Setup Kaggle API

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

# Download meta data used to generate dataset
The wikiart dataset wasn't balanced so we decided to generate our own dataset from a larger set of images from WikiArt.

We use 2 datasets for creating ours -
* Images taken from here - https://www.kaggle.com/c/painter-by-numbers/data
* Categories taken from here - https://github.com/cs-chan/ArtGAN/tree/master/WikiArt%20Dataset



Categories are based on Artist, Genre, Style

## Download classes from WikiArt

In [3]:
# ! mkdir -p /content/data/classes

In [4]:
# Get categories for artists, genres, styles
# file_link = "'https://drive.google.com/uc?export=download&id=1AuuWgyZnOB3mR8S7rXEVUrdzMTa6NaH1'"
# download_fpath = "'/content/data/classes/wikiart_classes.zip'"

# ! wget --no-check-certificate $file_link -O $download_fpath

In [5]:
# ! unzip /content/data/classes/wikiart_classes.zip -d /content/data/classes/

## Download files from painter-by-numbers

In [6]:
# Files from the Images dataset
! kaggle competitions files painter-by-numbers

name                                   size  creationDate         
------------------------------------  -----  -------------------  
train_7.zip                             4GB  2019-12-16 02:56:13  
train_2.zip                             4GB  2019-12-16 02:56:13  
train.zip                              36GB  2019-12-16 02:56:13  
train_5.zip                             4GB  2019-12-16 02:56:13  
solution_painter.csv                  261MB  2019-12-16 02:56:13  
train_6.zip                             4GB  2019-12-16 02:56:13  
train_4.zip                             4GB  2019-12-16 02:56:13  
test.zip                               11GB  2019-12-16 02:56:13  
train_3.zip                             4GB  2019-12-16 02:56:13  
sampleSubmission.csv                  261MB  2019-12-16 02:56:13  
all_data_info.csv                      13MB  2019-12-16 02:56:13  
replacements_for_corrupted_files.zip   17MB  2019-12-16 02:56:13  
train_1.zip                             5GB  2019-12-16 02:56:

In [7]:
! mkdir -p /content/data/raw
! kaggle competitions download painter-by-numbers -f train_info.csv -p /content/data/raw
! kaggle competitions download painter-by-numbers -f all_data_info.csv -p /content/data/raw

  0% 0.00/2.46M [00:00<?, ?B/s]
100% 2.46M/2.46M [00:00<00:00, 30.7MB/s]
 79% 3.00M/3.80M [00:00<00:00, 29.6MB/s]
100% 3.80M/3.80M [00:00<00:00, 27.8MB/s]


In [8]:
! unzip /content/data/raw/all_data_info.csv.zip -d /content/data/raw/
! unzip /content/data/raw/train_info.csv.zip -d /content/data/raw/

Archive:  /content/data/raw/all_data_info.csv.zip
  inflating: /content/data/raw/all_data_info.csv  
Archive:  /content/data/raw/train_info.csv.zip
  inflating: /content/data/raw/train_info.csv  


## Imports and load data

In [9]:
import os

import pandas as pd

In [10]:
data_dir = '/content/data'
raw_data_dir = os.path.join(data_dir, 'raw')

raw_df = pd.read_csv(os.path.join(raw_data_dir, 'all_data_info.csv'))
train_df = pd.read_csv(os.path.join(raw_data_dir, 'train_info.csv'))

# Explore the metadata a little

In [11]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103250 entries, 0 to 103249
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   artist        103250 non-null  object 
 1   date          76887 non-null   object 
 2   genre         101996 non-null  object 
 3   pixelsx       103250 non-null  float64
 4   pixelsy       103250 non-null  float64
 5   size_bytes    103250 non-null  float64
 6   source        103083 non-null  object 
 7   style         102264 non-null  object 
 8   title         103128 non-null  object 
 9   artist_group  103250 non-null  object 
 10  in_train      103250 non-null  bool   
 11  new_filename  103250 non-null  object 
dtypes: bool(1), float64(3), object(8)
memory usage: 8.8+ MB


In [12]:
raw_df.head()

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


In [13]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79433 entries, 0 to 79432
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  79433 non-null  object
 1   artist    79433 non-null  object
 2   title     79420 non-null  object
 3   style     78668 non-null  object
 4   genre     78540 non-null  object
 5   date      59178 non-null  object
dtypes: object(6)
memory usage: 3.6+ MB


In [14]:
train_df.head()

,filename,artist,title,style,genre,date
0,102257.jpg,5b39c876740bfc1cfaf544721c43cac3,Uriel,Color Field Painting,abstract,1955.0
1,75232.jpg,5b39c876740bfc1cfaf544721c43cac3,Vir Heroicus Sublimis,Color Field Painting,abstract,1950.0
2,29855.jpg,96e5bc98488ed589b9bf17ad9fd09371,Night March of a Hundred Demons (left half),Yamato-e,mythological painting,NaN
3,62252.jpg,5b39c876740bfc1cfaf544721c43cac3,"Who’s Afraid of Red, Yellow and Blue II",Color Field Painting,abstract,NaN
4,63861.jpg,5b39c876740bfc1cfaf544721c43cac3,Black Fire I,Color Field Painting,abstract,1963.0


# Preprocessing

In [15]:
raw_df['style'] = raw_df['style'].str.replace('Na.ve Art', 'Naive Art', regex=True)

In [16]:
# artists classes
artist_classes = list(raw_df.groupby(['artist']).size().sort_values(ascending=False)[:20].index)

# genre classes
genre_classes = list(raw_df.groupby(['genre']).size().sort_values(ascending=False)[:10].index)

# style classes
style_classes = list(raw_df.groupby(['style']).size().sort_values(ascending=False)[:20].index)

In [17]:
# write the classes out to text file
! mkdir /content/data/artists/
with open('/content/data/artists/classes.txt', 'w') as f:
    for artist in artist_classes:
        f.write(artist + '\n')

! mkdir /content/data/genres/
with open('/content/data/genres/classes.txt', 'w') as f:
    for genre in genre_classes:
        f.write(genre + '\n')

! mkdir /content/data/styles/
with open('/content/data/styles/classes.txt', 'w') as f:
    for style in style_classes:
        f.write(style + '\n')

# Make artists dataframe

In [18]:
# dataframe only consisting of chosen artists
artists_df = raw_df[raw_df['artist'].isin(artist_classes)]

# number of images per artist
artists_df.groupby(['artist']).size()

artist
Albrecht Durer                500
Boris Kustodiev               498
Camille Pissarro              499
Claude Monet                  498
Giovanni Battista Piranesi    500
Gustave Dore                  500
Ilya Repin                    500
Ivan Aivazovsky               500
Ivan Shishkin                 500
John Singer Sargent           500
Marc Chagall                  500
Martiros Saryan               499
Pablo Picasso                 500
Paul Cezanne                  499
Paul Gauguin                  498
Pierre-Auguste Renoir         500
Pyotr Konchalovsky            499
Raphael Kirchner              497
Rembrandt                     500
Zdislav Beksinski             500
dtype: int64

In [19]:
# save artist_df for later use
artists_df.to_csv('/content/data/artists/all_artists.csv')

# Make genres dataframe

In [20]:
raw_df_na_genre_dropped = raw_df[raw_df['genre'].notna()]

genres_df = raw_df_na_genre_dropped[raw_df_na_genre_dropped['genre'].isin(genre_classes)]

genres_df.groupby(['genre']).size()

genre
abstract               9498
cityscape              5348
genre painting        14260
illustration           3202
landscape             15006
portrait              16847
religious painting     7429
sketch and study       3644
still life             3132
symbolic painting      2545
dtype: int64

In [21]:
genres_df = genres_df.groupby(['genre']).sample(n=2000, random_state=42)
genres_df.groupby(['genre']).size()

genre
abstract              2000
cityscape             2000
genre painting        2000
illustration          2000
landscape             2000
portrait              2000
religious painting    2000
sketch and study      2000
still life            2000
symbolic painting     2000
dtype: int64

In [22]:
genres_df.to_csv('/content/data/genres/all_genres.csv')

# Make styles dataframe

In [23]:
raw_df_na_style_dropped = raw_df[raw_df['style'].notna()]

styles_df = raw_df_na_style_dropped[raw_df_na_style_dropped['style'].isin(style_classes)]

styles_df.groupby(['style']).size()

style
Abstract Expressionism           2074
Art Informel                     1267
Art Nouveau (Modern)             4899
Baroque                          4400
Cubism                           1747
Early Renaissance                1351
Expressionism                    7013
High Renaissance                 1314
Impressionism                   10643
Mannerism (Late Renaissance)     1342
Naive Art (Primitivism)          2295
Neoclassicism                    2038
Northern Renaissance             2379
Post-Impressionism               5778
Realism                         10523
Rococo                           2733
Romanticism                      9285
Surrealism                       4167
Symbolism                        3476
Ukiyo-e                          1426
dtype: int64

In [24]:
styles_df = styles_df.groupby(['style']).sample(n=1000, random_state=42)
styles_df.groupby(['style']).size()

style
Abstract Expressionism          1000
Art Informel                    1000
Art Nouveau (Modern)            1000
Baroque                         1000
Cubism                          1000
Early Renaissance               1000
Expressionism                   1000
High Renaissance                1000
Impressionism                   1000
Mannerism (Late Renaissance)    1000
Naive Art (Primitivism)         1000
Neoclassicism                   1000
Northern Renaissance            1000
Post-Impressionism              1000
Realism                         1000
Rococo                          1000
Romanticism                     1000
Surrealism                      1000
Symbolism                       1000
Ukiyo-e                         1000
dtype: int64

In [25]:
styles_df.to_csv('/content/data/styles/all_styles.csv')

In [26]:
# Download generated files
from google.colab import files

! zip -r /content/dataset.zip /content/data/

files.download('/content/dataset.zip')

  adding: content/data/ (stored 0%)
  adding: content/data/styles/ (stored 0%)
  adding: content/data/styles/all_styles.csv (deflated 73%)
  adding: content/data/styles/classes.txt (deflated 40%)
  adding: content/data/raw/ (stored 0%)
  adding: content/data/raw/train_info.csv (deflated 67%)
  adding: content/data/raw/all_data_info.csv (deflated 71%)
  adding: content/data/raw/all_data_info.csv.zip (stored 0%)
  adding: content/data/raw/train_info.csv.zip (stored 0%)
  adding: content/data/genres/ (stored 0%)
  adding: content/data/genres/all_genres.csv (deflated 71%)
  adding: content/data/genres/classes.txt (deflated 31%)
  adding: content/data/artists/ (stored 0%)
  adding: content/data/artists/classes.txt (deflated 30%)
  adding: content/data/artists/all_artists.csv (deflated 76%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>